In [11]:
import ee
import geemap
import ipywidgets as widgets
import ipyleaflet
from IPython.display import display

# -----------------------------------------------------------------------------
# Initialization
# -----------------------------------------------------------------------------
def initialize_earth_engine():
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize()

# -----------------------------------------------------------------------------
# Define region
# -----------------------------------------------------------------------------
def get_florida_geometry():
    return (ee.FeatureCollection("TIGER/2018/States")
            .filter(ee.Filter.eq('NAME', 'Florida'))
            .first()
            .geometry())

# -----------------------------------------------------------------------------
# Landsat parameter helper
# -----------------------------------------------------------------------------
def get_landsat_params(year, param):
    if 1985 <= year <= 2011:
        coll_id = 'LANDSAT/LT05/C02/T1_L2'
        bands = {
            'LST': 'ST_B6',
            'NDVI': ('SR_B4', 'SR_B3'),
            'NDBI': ('SR_B5', 'SR_B4')
        }
    elif 2012 <= year <= 2013:
        coll_id = 'LANDSAT/LE07/C02/T1_L2'
        bands = {
            'LST': 'ST_B6',
            'NDVI': ('SR_B4', 'SR_B3'),
            'NDBI': ('SR_B5', 'SR_B4')
        }
    elif 2014 <= year <= 2022:
        coll_id = 'LANDSAT/LC08/C02/T1_L2'
        bands = {
            'LST': 'ST_B10',
            'NDVI': ('SR_B5', 'SR_B4'),
            'NDBI': ('SR_B6', 'SR_B5')
        }
    else:
        coll_id = 'LANDSAT/LC09/C02/T1_L2'
        bands = {
            'LST': 'ST_B10',
            'NDVI': ('SR_B5', 'SR_B4'),
            'NDBI': ('SR_B6', 'SR_B5')
        }
    return coll_id, bands[param]

# -----------------------------------------------------------------------------
# Masking function without client-side getInfo in map
# -----------------------------------------------------------------------------
def apply_mask(image, use_radsat=False):
    # Mask clouds & shadows using QA_PIXEL bits
    masked = image.updateMask(
        image.select('QA_PIXEL').bitwiseAnd(0b11000).eq(0)
    )
    # Optionally mask saturated pixels
    if use_radsat:
        masked = masked.updateMask(
            masked.select('QA_RADSAT').eq(0)
        )
    return masked

# -----------------------------------------------------------------------------
# Image processing functions
# -----------------------------------------------------------------------------
def get_annual_lst(year):
    coll_id, band = get_landsat_params(year, 'LST')
    coll = (ee.ImageCollection(coll_id)
            .filterBounds(FLORIDA)
            .filterDate(f'{year}-01-01', f'{year}-12-31'))

    # Pull band list once to see if QA_RADSAT exists
    bands = coll.first().bandNames().getInfo()
    use_radsat = 'QA_RADSAT' in bands

    def to_celsius(img):
        return (img.select(band)
                   .multiply(0.00341802)
                   .add(149.0)
                   .subtract(273.15)
                   .rename('surface_temp'))

    return (coll
            .map(lambda img: apply_mask(img, use_radsat=use_radsat))
            .map(to_celsius)
            .median()
            .clip(FLORIDA)
            .selfMask())

def get_ndvi(year):
    coll_id, (nir, red) = get_landsat_params(year, 'NDVI')
    def compute_ndvi(img):
        return img.normalizedDifference([nir, red]).rename('NDVI')
    return (ee.ImageCollection(coll_id)
            .filterBounds(FLORIDA)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(lambda img: apply_mask(img))
            .map(compute_ndvi)
            .median()
            .clip(FLORIDA))

def get_ndbi(year):
    coll_id, (swir, nir) = get_landsat_params(year, 'NDBI')
    def compute_ndbi(img):
        return img.normalizedDifference([swir, nir]).rename('NDBI')
    return (ee.ImageCollection(coll_id)
            .filterBounds(FLORIDA)
            .filterDate(f'{year}-01-01', f'{year}-12-31')
            .map(lambda img: apply_mask(img))
            .map(compute_ndbi)
            .median()
            .clip(FLORIDA))

# -----------------------------------------------------------------------------
# Legend and Map Update (unchanged from your version)
# -----------------------------------------------------------------------------
def add_legend(vis, title):
    for c in m.controls:
        if getattr(c, 'is_legend', False):
            m.remove_control(c)
    html = f"<div style='padding:8px;background:rgba(255,255,255,0.8);'><b>{title}</b><br>"
    if title == 'LULC':
        for v, c, l in zip(NLCD_VALUES, NLCD_COLORS, NLCD_LABELS):
            html += f"<div style='display:flex;'><div style='width:18px;height:12px;background:#{c};margin-right:4px;border:1px solid #aaa;'></div>{v}: {l}</div>"
    else:
        palette = vis['palette']
        step = (vis['max'] - vis['min']) / len(palette)
        for i, col in enumerate(palette):
            lo, hi = vis['min'] + i*step, vis['min'] + (i+1)*step
            label = (
                f"≤ {hi:.2f}" if i == 0 else
                f"> {lo:.2f}" if i == len(palette)-1 else
                f"{lo:.2f} – {hi:.2f}"
            )
            html += f"<div style='display:flex;'><div style='width:18px;height:12px;background:#{col};margin-right:4px;border:1px solid #aaa;'></div>{label}</div>"
    html += "</div>"
    widget = widgets.HTML(html)
    ctrl = ipyleaflet.WidgetControl(widget=widget, position='bottomright')
    ctrl.is_legend = True
    m.add_control(ctrl)

def update_map(*_):
    global _prev
    if _prev:
        m.remove_layer(_prev)

    p, y = param_dd.value, int(year_dd.value)
    info_lbl.value = f'Loading {p} {y}…'
    if p == 'LST':
        img, vis, name = get_annual_lst(y), thermal_vis, f'LST {y}'
    elif p == 'UTFVI':
        lst = get_annual_lst(y)
        mean = ee.Number(lst.reduceRegion(ee.Reducer.mean(), FLORIDA, 1000).get('surface_temp'))
        img, vis, name = lst.subtract(mean).divide(mean).rename('UTFVI'), utfvi_vis, f'UTFVI {y}'
    elif p == 'Hotspots':
        lst = get_annual_lst(y)
        stats = lst.reduceRegion(ee.Reducer.mean().combine(ee.Reducer.stdDev(), True),
                                 FLORIDA, 1000)
        mean, std = ee.Number(stats.get('surface_temp_mean')), ee.Number(stats.get('surface_temp_stdDev'))
        img = lst.gt(mean.add(std.multiply(2))).rename('hotspot')
        vis, name = hotspot_vis, f'Hotspots {y}'
    elif p == 'NDVI':
        img, vis, name = get_ndvi(y), ndvi_vis, f'NDVI {y}'
    elif p == 'NDBI':
        img, vis, name = get_ndbi(y), ndbi_vis, f'NDBI {y}'
    else:
        img = NLCD_COLLECTION.filter(ee.Filter.eq('system:index', str(y))).first().select('b1').clip(FLORIDA)
        vis, name = {'min':11, 'max':95, 'palette':NLCD_COLORS}, f'LULC {y}'

    _prev = m.add_layer(img, vis, name)
    add_legend(vis, 'LULC' if p == 'LULC' else p)
    info_lbl.value = f'{p} {y} loaded'

def on_param_change(change):
    if change.new == 'LULC':
        year_dd.options = NLCD_YEARS
        year_dd.value = NLCD_YEARS[-1]
    else:
        year_dd.options = LST_YEARS
        year_dd.value = max(LST_YEARS)
    update_map()

# -----------------------------------------------------------------------------
# Main execution
# -----------------------------------------------------------------------------
initialize_earth_engine()
FLORIDA = get_florida_geometry()
LST_YEARS = list(range(1985, 2025))
NLCD_COLLECTION = ee.ImageCollection("projects/sat-io/open-datasets/USGS/ANNUAL_NLCD/LANDCOVER")
NLCD_YEARS = NLCD_COLLECTION.aggregate_array("system:index").distinct().sort().getInfo()

NLCD_VALUES = [11,12,21,22,23,24,31,41,42,43,52,71,81,82,90,95]
NLCD_COLORS = ['466b9f','d1def8','dec5c5','d99282','eb0000','ab0000',
               'b3ac9f','68ab5f','1c5f2c','b5c58f','ccb879','dfdfc2',
               'dcd939','ab6c28','b8d9eb','6c9fb8']
NLCD_LABELS = ['Open Water','Perennial Ice/Snow','Dev, Open Space','Dev, Low Intensity',
               'Dev, Medium Intensity','Dev, High Intensity','Barren Land',
               'Deciduous Forest','Evergreen Forest','Mixed Forest','Shrub/Scrub',
               'Grassland/Herbaceous','Pasture/Hay','Cultivated Crops',
               'Woody Wetlands','Emergent Herbaceous Wetlands']

thermal_vis = {'min': -3, 'max': 37,
               'palette': ['000080','0000FF','00FFFF','00FF00','FFFF00','FFA500','FF0000']}
utfvi_vis = {'min': -0.3, 'max': 0.3, 'palette': ['0000FF','FFFFFF','FF0000']}
hotspot_vis = {'min': 0, 'max': 1, 'palette': ['00000000','FF0000']}
ndvi_vis = {'min': -0.2, 'max': 1,
            'palette': ['FFFFFF','CE7E45','FCD163','66A000','207401']}
ndbi_vis = {'min': -0.5, 'max': 0.5,
            'palette': ['FFFFFF','B4B4B4','999999','7F7F7F','666666','4C4C4C','333333']}

# Widgets & map
m = geemap.Map(center=[28.0, -82.4], zoom=7)
m.addLayer(FLORIDA, {'color': 'white', 'fillColor': '00000000'}, 'Florida')

param_dd = widgets.Dropdown(
    options=['LST','UTFVI','Hotspots','NDVI','NDBI','LULC'],
    value='LST',
    description='Parameter:'
)
year_dd = widgets.Dropdown(options=LST_YEARS, value=2024, description='Year:')
info_lbl = widgets.Label()
_prev = None

param_dd.observe(on_param_change, names='value')
year_dd.observe(update_map, names='value')

display(widgets.HBox([param_dd, year_dd]), info_lbl)
display(m)
update_map()


Label(value='')

Map(center=[28.0, -82.4], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…

To Create Box Plots for NDVI, NDBI and LST:

In [12]:
import ee
import geemap
import ipywidgets as widgets
import ipyleaflet
from IPython.display import display, FileLink
import requests
import os

# -----------------------------------------------------------------------------
# Initialization
# -----------------------------------------------------------------------------
def initialize_earth_engine():
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize()

# -----------------------------------------------------------------------------
# Load state and county collections
# -----------------------------------------------------------------------------
STATE_FC  = ee.FeatureCollection("TIGER/2018/States")
COUNTY_FC = ee.FeatureCollection("TIGER/2018/Counties")

_state_info  = STATE_FC.getInfo()['features']
STATE_NAMES  = sorted(f['properties']['NAME'] for f in _state_info)
STATE_FP_MAP = {f['properties']['NAME']: f['properties']['STATEFP'] for f in _state_info}

# -----------------------------------------------------------------------------
# Visualization parameters (no 'crs' keys)
# -----------------------------------------------------------------------------
thermal_vis = {
    'min': -10, 'max': 40,
    'palette': ['blue', 'green', 'yellow', 'red']
}
ndvi_vis = {
    'min': 0, 'max': 1,
    'palette': ['white', 'lightgreen', 'darkgreen']
}
ndbi_vis = {
    'min': -1, 'max': 1,
    'palette': ['brown', 'white', 'blue']
}
utfvi_vis = {
    'min': -0.5, 'max': 0.5,
    'palette': ['green', 'white', 'red']
}
hotspot_vis = {
    'min': 0, 'max': 1,
    'palette': ['lightgray', 'red']
}

# -----------------------------------------------------------------------------
# Landsat band parameters
# -----------------------------------------------------------------------------
def get_landsat_params(year, param):
    if 1985 <= year <= 2011:
        coll = 'LANDSAT/LT05/C02/T1_L2'
        bands = {'LST': 'ST_B6', 'NDVI': ('SR_B4', 'SR_B3'), 'NDBI': ('SR_B5', 'SR_B4')}
    elif 2012 <= year <= 2013:
        coll = 'LANDSAT/LE07/C02/T1_L2'
        bands = {'LST': 'ST_B6', 'NDVI': ('SR_B4', 'SR_B3'), 'NDBI': ('SR_B5', 'SR_B4')}
    elif 2014 <= year <= 2022:
        coll = 'LANDSAT/LC08/C02/T1_L2'
        bands = {'LST': 'ST_B10', 'NDVI': ('SR_B5', 'SR_B4'), 'NDBI': ('SR_B6', 'SR_B5')}
    else:
        coll = 'LANDSAT/LC09/C02/T1_L2'
        bands = {'LST': 'ST_B10', 'NDVI': ('SR_B5', 'SR_B4'), 'NDBI': ('SR_B6', 'SR_B5')}
    return coll, bands[param]

# -----------------------------------------------------------------------------
# Cloud and radiometric mask
# -----------------------------------------------------------------------------
def apply_mask(img, use_radsat=False):
    qa = img.select('QA_PIXEL').bitwiseAnd(0b11000).eq(0)
    masked = img.updateMask(qa)
    if use_radsat:
        flag = masked.bandNames().contains('QA_RADSAT')
        masked = ee.Image(ee.Algorithms.If(
            flag,
            masked.updateMask(masked.select('QA_RADSAT').eq(0)),
            masked
        ))
    return masked

# -----------------------------------------------------------------------------
# Time-series functions
# -----------------------------------------------------------------------------
def get_annual_lst(year, region):
    coll, band = get_landsat_params(year, 'LST')
    def to_celsius(i):
        return i.select(band) \
                .multiply(0.00341802) \
                .add(149.0) \
                .subtract(273.15) \
                .rename('surface_temp')
    return (
        ee.ImageCollection(coll)
          .filterBounds(region)
          .filterDate(f'{year}-01-01', f'{year}-12-31')
          .map(lambda i: apply_mask(i, True))
          .map(to_celsius)
          .median()
          .clip(region)
          .selfMask()
    )

def get_ndvi(year, region):
    coll, (nir, red) = get_landsat_params(year, 'NDVI')
    return (
        ee.ImageCollection(coll)
          .filterBounds(region)
          .filterDate(f'{year}-01-01', f'{year}-12-31')
          .map(lambda i: apply_mask(i))
          .map(lambda i: i.normalizedDifference([nir, red]).rename('NDVI'))
          .median()
          .clip(region)
    )

def get_ndbi(year, region):
    coll, (swir, nir) = get_landsat_params(year, 'NDBI')
    return (
        ee.ImageCollection(coll)
          .filterBounds(region)
          .filterDate(f'{year}-01-01', f'{year}-12-31')
          .map(lambda i: apply_mask(i))
          .map(lambda i: i.normalizedDifference([swir, nir]).rename('NDBI'))
          .median()
          .clip(region)
    )

# -----------------------------------------------------------------------------
# Legend utility
# -----------------------------------------------------------------------------
def add_legend(vis, title):
    # remove old legend
    for c in m.controls:
        if getattr(c, 'is_legend', False):
            m.remove_control(c)
    # build new legend
    html = f"<div style='padding:8px;background:rgba(255,255,255,0.8);'><b>{title}</b><br>"
    pal, st, mx = vis['palette'], vis['min'], vis['max']
    step = (mx - st) / len(pal)
    for i, col in enumerate(pal):
        lo = st + i * step
        hi = st + (i + 1) * step
        if i == 0:
            lbl = f"≤{hi:.2f}"
        elif i == len(pal) - 1:
            lbl = f">{lo:.2f}"
        else:
            lbl = f"{lo:.2f}-{hi:.2f}"
        html += f"<div style='display:flex;'><div style='width:18px;height:12px;background:{col};'></div>{lbl}</div>"
    html += '</div>'
    w = widgets.HTML(html)
    ctrl = ipyleaflet.WidgetControl(widget=w, position='bottomright')
    ctrl.is_legend = True
    m.add_control(ctrl)

# -----------------------------------------------------------------------------
# Update counties dropdown
# -----------------------------------------------------------------------------
def update_counties(change):
    state = change.new if hasattr(change, 'new') else state_dd.value
    fips  = STATE_FP_MAP[state]
    names = (COUNTY_FC
             .filter(ee.Filter.eq('STATEFP', fips))
             .aggregate_array('NAME')
             .distinct()
             .sort()
             .getInfo())
    county_dd.options = names
    county_dd.value   = names[0]

# -----------------------------------------------------------------------------
# Update map and clip to selected region
# -----------------------------------------------------------------------------
_prev_layer = None
def update_map(*_):
    global _prev_layer, REGION

    if _prev_layer:
        m.remove_layer(_prev_layer)

    # choose state or county feature
    if region_type_dd.value == 'State':
        feat = STATE_FC.filter(ee.Filter.eq('NAME', state_dd.value)).first()
    else:
        fips = STATE_FP_MAP[state_dd.value]
        feat = (COUNTY_FC
                .filter(ee.Filter.eq('STATEFP', fips))
                .filter(ee.Filter.eq('NAME', county_dd.value))
                .first())
    REGION = feat.geometry()

    yr = int(year_dd.value)
    p  = param_dd.value

    # compute image + visualization
    if p == 'LST':
        img, vis = get_annual_lst(yr, REGION), thermal_vis

    elif p == 'NDVI':
        img, vis = get_ndvi(yr, REGION), ndvi_vis

    elif p == 'NDBI':
        img, vis = get_ndbi(yr, REGION), ndbi_vis

    elif p == 'UTFVI':
        lst = get_annual_lst(yr, REGION)
        mv  = lst.reduceRegion(ee.Reducer.mean(), REGION, scale_dd.value).get('surface_temp')
        img = lst.subtract(ee.Number(mv)).divide(ee.Number(mv)).rename('UTFVI')
        vis = utfvi_vis

    else:  # Hotspots
        lst   = get_annual_lst(yr, REGION)
        stats = lst.reduceRegion(
                    ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True),
                    REGION, scale_dd.value
                )
        mv = stats.get('surface_temp_mean')
        sd = stats.get('surface_temp_stdDev')
        img = lst.gt(ee.Number(mv).add(ee.Number(sd).multiply(2))).rename('Hotspot')
        vis = hotspot_vis

    # force-clip to chosen region
    img = img.clip(REGION)

    # add to map
    _prev_layer = m.add_layer(img, vis, f"{p} {yr}")
    add_legend(vis, p)
    m.centerObject(feat, zoom=8)
    info_lbl.value = f"{p} {yr} (clipped to {region_type_dd.value})"

# -----------------------------------------------------------------------------
# Download map button callback
# -----------------------------------------------------------------------------
def download_map(b):
    yr = int(year_dd.value)
    p  = param_dd.value
    img, vis = (
        (get_annual_lst(yr, REGION), thermal_vis)
        if p == 'LST'
        else (get_ndvi(yr, REGION), ndvi_vis)
    )
    url = img.getThumbURL({
        'min':      vis['min'],
        'max':      vis['max'],
        'palette':  vis['palette'],
        'region':   REGION,
        'dimensions': resolution_dd.value,
        'format':   'png'
    })
    resp = requests.get(url)
    fn   = f"map_{p}_{state_dd.value}_{region_type_dd.value}_" \
           f"{(county_dd.value if region_type_dd.value=='County' else state_dd.value)}_{yr}.png"
    with open(fn, 'wb') as f:
        f.write(resp.content)
    info_lbl.value = f"Saved {fn}"
    display(FileLink(fn))

# -----------------------------------------------------------------------------
# Animation callback
# -----------------------------------------------------------------------------
def download_animation(b):
    start, end = int(anim_start_dd.value), int(anim_end_dd.value)
    yrs = list(range(start, end + 1, step_dd.value))
    frames = []
    for yr in yrs:
        if param_dd.value == 'LST':
            frames.append(get_annual_lst(yr, REGION).visualize(**thermal_vis))
        else:
            frames.append(get_ndvi(yr, REGION).visualize(**ndvi_vis))
    col = ee.ImageCollection(frames)
    url = col.getVideoThumbURL({
        'dimensions':      resolution_dd.value,
        'region':          REGION,
        'framesPerSecond': 1,
        'format':          'gif',
        'backgroundColor': 'white'
    })
    resp = requests.get(url)
    fn   = f"anim_{param_dd.value}_{state_dd.value}_{region_type_dd.value}_" \
           f"{(county_dd.value if region_type_dd.value=='County' else state_dd.value)}_" \
           f"{start}_{end}.gif"
    with open(fn, 'wb') as f:
        f.write(resp.content)
    info_lbl.value = f"Saved {fn}"
    display(FileLink(fn))

# -----------------------------------------------------------------------------
# Widget setup & display
# -----------------------------------------------------------------------------
initialize_earth_engine()

year_dd        = widgets.Dropdown(options=list(range(1985,2025)),
                                  value=2024, description='Year:')
param_dd       = widgets.Dropdown(options=['LST','NDVI','NDBI','UTFVI','Hotspots'],
                                  value='LST', description='Parameter:')
state_dd       = widgets.Dropdown(options=STATE_NAMES,
                                  value='Florida', description='State:')
region_type_dd = widgets.Dropdown(options=['State','County'],
                                  value='State', description='Region Type:')
county_dd      = widgets.Dropdown(options=[], description='County:')
resolution_dd  = widgets.IntText(value=256, description='Resolution(px):')
scale_dd       = widgets.IntText(value=1000, description='Scale(m):')
download_map_btn  = widgets.Button(description='Download Map')
anim_start_dd     = widgets.Dropdown(options=list(range(1985,2025)),
                                     value=1985, description='Anim Start:')
anim_end_dd       = widgets.Dropdown(options=list(range(1985,2025)),
                                     value=2024, description='Anim End:')
step_dd           = widgets.IntSlider(value=1, min=1, max=10,
                                      description='Step(yrs):')
animate_btn       = widgets.Button(description='Download Animation')
info_lbl          = widgets.Label()

state_dd.observe(update_counties, names='value')
for w in [param_dd, year_dd, region_type_dd, county_dd]:
    w.observe(update_map, names='value')
download_map_btn.on_click(download_map)
animate_btn.on_click(download_animation)

update_counties(type('x', (), {'new': state_dd.value}))
map_controls  = widgets.HBox([param_dd, year_dd, state_dd,
                              region_type_dd, county_dd,
                              resolution_dd, download_map_btn])
anim_controls = widgets.HBox([step_dd, anim_start_dd,
                              anim_end_dd, animate_btn])
ui = widgets.VBox([map_controls, anim_controls, info_lbl])

m = geemap.Map()
display(ui, m)

_prev_layer = None
update_map()


TransportError: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))

In [ ]:
import ee
import geemap
import ipywidgets as widgets
import ipyleaflet
from ipyleaflet import LayersControl
from IPython.display import display, FileLink
import requests
import os

# -----------------------------------------------------------------------------
# 1. Initialize Earth Engine
# -----------------------------------------------------------------------------
def initialize_earth_engine():
    try:
        ee.Initialize()
    except Exception:
        ee.Authenticate()
        ee.Initialize()

# -----------------------------------------------------------------------------
# 2. Load state & county FeatureCollections
# -----------------------------------------------------------------------------
STATE_FC  = ee.FeatureCollection("TIGER/2018/States")
COUNTY_FC = ee.FeatureCollection("TIGER/2018/Counties")

_state_info  = STATE_FC.getInfo()['features']
STATE_NAMES  = sorted(f['properties']['NAME'] for f in _state_info)
STATE_FP_MAP = {f['properties']['NAME']: f['properties']['STATEFP'] for f in _state_info}

# -----------------------------------------------------------------------------
# 3. Visualization parameter dicts (no 'crs' keys)
# -----------------------------------------------------------------------------
thermal_vis = {'min': -10, 'max': 40,   'palette': ['blue','green','yellow','red']}
ndvi_vis    = {'min':   0, 'max':  1,   'palette': ['white','lightgreen','darkgreen']}
ndbi_vis    = {'min':  -1, 'max':  1,   'palette': ['brown','white','blue']}
utfvi_vis   = {'min': -0.5,'max': 0.5,  'palette': ['green','white','red']}
hotspot_vis = {'min':   0, 'max':  1,   'palette': ['lightgray','red']}

# -----------------------------------------------------------------------------
# 4. Landsat band selection
# -----------------------------------------------------------------------------
def get_landsat_params(year, param):
    if   1985 <= year <= 2011:
        coll  = 'LANDSAT/LT05/C02/T1_L2'
        bands = {'LST':'ST_B6','NDVI':('SR_B4','SR_B3'),'NDBI':('SR_B5','SR_B4')}
    elif 2012 <= year <= 2013:
        coll  = 'LANDSAT/LE07/C02/T1_L2'
        bands = {'LST':'ST_B6','NDVI':('SR_B4','SR_B3'),'NDBI':('SR_B5','SR_B4')}
    elif 2014 <= year <= 2022:
        coll  = 'LANDSAT/LC08/C02/T1_L2'
        bands = {'LST':'ST_B10','NDVI':('SR_B5','SR_B4'),'NDBI':('SR_B6','SR_B5')}
    else:
        coll  = 'LANDSAT/LC09/C02/T1_L2'
        bands = {'LST':'ST_B10','NDVI':('SR_B5','SR_B4'),'NDBI':('SR_B6','SR_B5')}
    return coll, bands[param]

# -----------------------------------------------------------------------------
# 5. Cloud & radiometric mask
# -----------------------------------------------------------------------------
def apply_mask(img, use_radsat=False):
    qa = img.select('QA_PIXEL').bitwiseAnd(0b11000).eq(0)
    masked = img.updateMask(qa)
    if use_radsat:
        flag = masked.bandNames().contains('QA_RADSAT')
        masked = ee.Image(ee.Algorithms.If(
            flag,
            masked.updateMask(masked.select('QA_RADSAT').eq(0)),
            masked
        ))
    return masked

# -----------------------------------------------------------------------------
# 6. Time-series compositing functions
# -----------------------------------------------------------------------------
def get_annual_lst(year, region):
    coll, band = get_landsat_params(year, 'LST')
    def to_celsius(image):
        return (image.select(band)
                     .multiply(0.00341802)
                     .add(149.0)
                     .subtract(273.15)
                     .rename('surface_temp'))
    return (ee.ImageCollection(coll)
              .filterBounds(region)
              .filterDate(f'{year}-01-01', f'{year}-12-31')
              .map(lambda i: apply_mask(i, True))
              .map(to_celsius)
              .median()
              .clip(region)
              .selfMask())

def get_ndvi(year, region):
    coll, (nir, red) = get_landsat_params(year, 'NDVI')
    return (ee.ImageCollection(coll)
              .filterBounds(region)
              .filterDate(f'{year}-01-01', f'{year}-12-31')
              .map(lambda i: apply_mask(i))
              .map(lambda i: i.normalizedDifference([nir, red]).rename('NDVI'))
              .median()
              .clip(region))

def get_ndbi(year, region):
    coll, (swir, nir) = get_landsat_params(year, 'NDBI')
    return (ee.ImageCollection(coll)
              .filterBounds(region)
              .filterDate(f'{year}-01-01', f'{year}-12-31')
              .map(lambda i: apply_mask(i))
              .map(lambda i: i.normalizedDifference([swir, nir]).rename('NDBI'))
              .median()
              .clip(region))

# -----------------------------------------------------------------------------
# 7. Legend helper
# -----------------------------------------------------------------------------
def add_legend(vis, title):
    # remove old
    for ctrl in m.controls:
        if getattr(ctrl, 'is_legend', False):
            m.remove_control(ctrl)
    # build new
    html = f"<div style='padding:8px;background:rgba(255,255,255,0.8);'><b>{title}</b><br>"
    pal, st, mx = vis['palette'], vis['min'], vis['max']
    step = (mx - st) / len(pal)
    for i, col in enumerate(pal):
        lo, hi = st + i*step, st + (i+1)*step
        lbl = "≤{:.2f}".format(hi) if i==0 else (">{:.2f}".format(lo) if i==len(pal)-1 else "{:.2f}-{:.2f}".format(lo,hi))
        html += f"<div style='display:flex;'><div style='width:18px;height:12px;background:{col};'></div>{lbl}</div>"
    html += "</div>"
    widget = widgets.HTML(html)
    legend_ctrl = ipyleaflet.WidgetControl(widget=widget, position='bottomright')
    legend_ctrl.is_legend = True
    m.add_control(legend_ctrl)

# -----------------------------------------------------------------------------
# 8. County dropdown updater
# -----------------------------------------------------------------------------
def update_counties(change):
    state = change.new if hasattr(change, 'new') else state_dd.value
    fips  = STATE_FP_MAP[state]
    names = (COUNTY_FC
             .filter(ee.Filter.eq('STATEFP', fips))
             .aggregate_array('NAME')
             .distinct()
             .sort()
             .getInfo())
    county_dd.options = names
    county_dd.value   = names[0]

# -----------------------------------------------------------------------------
# 9. Map update (clip to selected region)
# -----------------------------------------------------------------------------
_prev_layer = None
def update_map(*_):
    global _prev_layer, REGION
    if _prev_layer:
        m.remove_layer(_prev_layer)

    # choose feature
    if region_type_dd.value == 'State':
        feat = STATE_FC.filter(ee.Filter.eq('NAME', state_dd.value)).first()
    else:
        fips = STATE_FP_MAP[state_dd.value]
        feat = (COUNTY_FC
                .filter(ee.Filter.eq('STATEFP', fips))
                .filter(ee.Filter.eq('NAME', county_dd.value))
                .first())

    REGION = feat.geometry()
    yr, p = int(year_dd.value), param_dd.value

    # compute image & vis
    if p == 'LST':
        img, vis = get_annual_lst(yr, REGION), thermal_vis
    elif p == 'NDVI':
        img, vis = get_ndvi(yr, REGION),    ndvi_vis
    elif p == 'NDBI':
        img, vis = get_ndbi(yr, REGION),    ndbi_vis
    elif p == 'UTFVI':
        lst = get_annual_lst(yr, REGION)
        mv  = lst.reduceRegion(ee.Reducer.mean(), REGION, scale_dd.value).get('surface_temp')
        img = lst.subtract(ee.Number(mv)).divide(ee.Number(mv)).rename('UTFVI')
        vis = utfvi_vis
    else:  # Hotspots
        lst   = get_annual_lst(yr, REGION)
        stats = lst.reduceRegion(
            ee.Reducer.mean().combine(ee.Reducer.stdDev(), None, True),
            REGION, scale_dd.value
        )
        mv = stats.get('surface_temp_mean')
        sd = stats.get('surface_temp_stdDev')
        img = lst.gt(ee.Number(mv).add(ee.Number(sd).multiply(2))).rename('Hotspot')
        vis = hotspot_vis

    # clip & add
    img = img.clip(REGION)
    _prev_layer = m.add_layer(img, vis, f"{p} {yr}")
    add_legend(vis, p)
    m.centerObject(feat, zoom=8)
    info_lbl.value = f"{p} {yr} (clipped to {region_type_dd.value})"

# -----------------------------------------------------------------------------
# 10. Download callbacks
# -----------------------------------------------------------------------------
def download_map(b):
    yr, p = int(year_dd.value), param_dd.value
    img, vis = (get_annual_lst(yr, REGION), thermal_vis) if p=='LST' else (get_ndvi(yr, REGION), ndvi_vis)
    url = img.getThumbURL({
        'min': vis['min'], 'max': vis['max'],
        'palette': vis['palette'],
        'region': REGION,
        'dimensions': resolution_dd.value,
        'format': 'png'
    })
    resp = requests.get(url)
    fn   = f"map_{p}_{state_dd.value}_{region_type_dd.value}_" \
           f"{(county_dd.value if region_type_dd.value=='County' else state_dd.value)}_{yr}.png"
    with open(fn, 'wb') as f:
        f.write(resp.content)
    info_lbl.value = f"Saved {fn}"
    display(FileLink(fn))

def download_animation(b):
    start, end = int(anim_start_dd.value), int(anim_end_dd.value)
    yrs         = list(range(start, end+1, step_dd.value))
    frames      = []
    for yr in yrs:
        if param_dd.value=='LST':
            frames.append(get_annual_lst(yr, REGION).visualize(**thermal_vis))
        else:
            frames.append(get_ndvi(yr, REGION).visualize(**ndvi_vis))
    col = ee.ImageCollection(frames)
    url = col.getVideoThumbURL({
        'dimensions': resolution_dd.value,
        'region':      REGION,
        'framesPerSecond': 1,
        'format': 'gif',
        'backgroundColor': 'white'
    })
    resp = requests.get(url)
    fn   = f"anim_{param_dd.value}_{state_dd.value}_{region_type_dd.value}_" \
           f"{(county_dd.value if region_type_dd.value=='County' else state_dd.value)}_{start}_{end}.gif"
    with open(fn, 'wb') as f:
        f.write(resp.content)
    info_lbl.value = f"Saved {fn}"
    display(FileLink(fn))

# -----------------------------------------------------------------------------
# 11. Widget setup & callbacks
# -----------------------------------------------------------------------------
initialize_earth_engine()

year_dd        = widgets.Dropdown(options=list(range(1985,2025)), value=2024, description='Year:')
param_dd       = widgets.Dropdown(options=['LST','NDVI','NDBI','UTFVI','Hotspots'], value='LST', description='Parameter:')
state_dd       = widgets.Dropdown(options=STATE_NAMES, value='Florida', description='State:')
region_type_dd = widgets.Dropdown(options=['State','County'], value='State', description='Region Type:')
county_dd      = widgets.Dropdown(options=[], description='County:')
resolution_dd  = widgets.IntText(value=256, description='Resolution(px):')
scale_dd       = widgets.IntText(value=1000, description='Scale(m):')
download_map_btn = widgets.Button(description='Download Map')
anim_start_dd    = widgets.Dropdown(options=list(range(1985,2025)), value=1985, description='Anim Start:')
anim_end_dd      = widgets.Dropdown(options=list(range(1985,2025)), value=2024, description='Anim End:')
step_dd          = widgets.IntSlider( value=1, min=1, max=10, description='Step(yrs):')
animate_btn      = widgets.Button(description='Download Animation')
info_lbl         = widgets.Label()

state_dd.observe(update_counties, names='value')
for w in [param_dd, year_dd, region_type_dd, county_dd]:
    w.observe(update_map, names='value')
download_map_btn.on_click(download_map)
animate_btn.on_click(download_animation)

# initialize county list
update_counties(type('x', (), {'new': state_dd.value}))

# build UI layout
map_controls  = widgets.HBox([param_dd, year_dd, state_dd, region_type_dd, county_dd, resolution_dd, download_map_btn])
anim_controls = widgets.HBox([step_dd, anim_start_dd, anim_end_dd, animate_btn])
ui            = widgets.VBox([map_controls, anim_controls, info_lbl])

# -----------------------------------------------------------------------------
# 12. Create map widget & add EE county boundaries layer
# -----------------------------------------------------------------------------
m = geemap.Map()
styled_counties = COUNTY_FC.style(**{
    'color':     '0000FF',    # blue outline
    'fillColor': '00000000',  # transparent fill
    'width':     1
})
m.add_layer(styled_counties, {}, 'County Boundaries')
m.add_control(LayersControl(position='topright'))

# -----------------------------------------------------------------------------
# 13. Display UI + map, then draw first layer
# -----------------------------------------------------------------------------
display(ui, m)
_prev_layer = None
update_map()


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Import required libraries
import ee
import geemap
import pandas as pd

# Initialize Earth Engine
try:
    ee.Initialize()
    print("Earth Engine initialized successfully!")
except Exception as e:
    print(f"Earth Engine initialization failed: {e}")
    # For Colab, you might need to authenticate first
    # If running in Colab, uncomment the following lines:
    # ee.Authenticate()
    # ee.Initialize()
    print("Please authenticate and initialize Earth Engine.")

# Define the area of interest: Palm Beach County, Florida
# Load TIGER county data
counties = ee.FeatureCollection("TIGER/2018/Counties")

# Filter for Florida counties (STATEFP = '12') and then Palm Beach County
florida_counties = counties.filter(ee.Filter.eq('STATEFP', '12'))
palm_beach_county = florida_counties.filter(ee.Filter.eq('NAME', 'Palm Beach'))

# Get the geometry of Palm Beach County
palm_beach_geometry = palm_beach_county.geometry()

# Load NLCD land cover data for 1985 using the correct path
nlcd_collection = ee.ImageCollection("projects/sat-io/open-datasets/USGS/ANNUAL_NLCD/LANDCOVER")
nlcd_1985 = nlcd_collection.filter(ee.Filter.eq('system:index', '1985')).first().select('landcover')

# Define the developed and barren land classes
developed_classes = [21, 22, 23, 24]  # Developed classes
barren_class = [31]  # Barren land class

# Create a mask for developed and barren land
developed_mask = nlcd_1985.eq(21).Or(nlcd_1985.eq(22)).Or(nlcd_1985.eq(23)).Or(nlcd_1985.eq(24))
barren_mask = nlcd_1985.eq(31)

# Combine developed and barren masks
developed_barren_mask = developed_mask.Or(barren_mask)

# Calculate total area of Palm Beach County
# Use a scale of 30 meters (typical for NLCD)
total_area_image = ee.Image.constant(1).clip(palm_beach_geometry)
total_area = total_area_image.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=palm_beach_geometry,
    scale=30,
    maxPixels=1e9
).get('constant')

# Calculate developed and barren area
developed_barren_area = developed_barren_mask.multiply(ee.Image.pixelArea()).reduceRegion(
    reducer=ee.Reducer.sum(),
    geometry=palm_beach_geometry,
    scale=30,
    maxPixels=1e9
).get('landcover')

# Calculate undeveloped area
undeveloped_area = ee.Number(total_area).subtract(ee.Number(developed_barren_area))

# Convert square meters to square miles (1 square meter = 3.86102e-7 square miles)
total_area_sqmi = ee.Number(total_area).multiply(3.86102e-7)
developed_barren_area_sqmi = ee.Number(developed_barren_area).multiply(3.86102e-7)
undeveloped_area_sqmi = ee.Number(undeveloped_area).multiply(3.86102e-7)

# Get the information
total_area_value = total_area_sqmi.getInfo()
developed_barren_area_value = developed_barren_area_sqmi.getInfo()
undeveloped_area_value = undeveloped_area_sqmi.getInfo()

# Print the results
print("\nLand Use Land Cover Analysis for Palm Beach County, Florida (1985)")
print("=" * 60)
print(f"Total County Area: {total_area_value:.2f} square miles")
print(f"Developed + Barren Area: {developed_barren_area_value:.2f} square miles")
print(f"Undeveloped Area: {undeveloped_area_value:.2f} square miles")
print("=" * 60)

# Create a simple table of results
results = {
    'Category': ['Total County Area', 'Developed + Barren Area', 'Undeveloped Area'],
    'Area (sq mi)': [total_area_value, developed_barren_area_value, undeveloped_area_value]
}

results_df = pd.DataFrame(results)
print("\nResults Table:")
print(results_df.to_string(index=False))

# Optional: Calculate and print percentage of developed and barren land
developed_barren_percent = (developed_barren_area_value / total_area_value) * 100
print(f"\nPercentage of Developed + Barren Land: {developed_barren_percent:.2f}%")
print(f"Percentage of Undeveloped Land: {100 - developed_barren_percent:.2f}%")

Earth Engine initialized successfully!


EEException: Image.select: Parameter 'input' is required and may not be null.